In [1]:
import pandas as pd
import numpy as np

In [147]:
df = pd.read_csv('/content/drive/MyDrive/csv/vgames2.csv')
df.drop(columns='Unnamed: 0', axis=1, inplace=True)

df1 = df 

df1['Year'] = df1['Year'].fillna(2005)
df1['Genre'] = df1['Genre'].fillna('etc')
df1

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,Candace Kane's Candy Factory,DS,2008.0,Action,Destineer,0.04,0,0,0
1,The Munchables,Wii,2009.0,Action,Namco Bandai Games,0.17,0,0,0.01
2,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010.0,Adventure,Alchemist,0,0,0.02,0
3,Deal or No Deal: Special Edition,DS,2010.0,Misc,Zoo Games,0.04,0,0,0
4,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010.0,Platform,D3Publisher,0.12,0.09,0,0.04
...,...,...,...,...,...,...,...,...,...
16593,Ice Age 2: The Meltdown,GC,2006.0,Platform,Vivendi Games,0.15,0.04,0,0.01
16594,Rainbow Islands: Revolution,PSP,2005.0,Action,Rising Star Games,0.01,0,0,0
16595,NBA 2K16,PS3,2015.0,Sports,Take-Two Interactive,0.44,0.19,0.03,0.13
16596,Toukiden: The Age of Demons,PSV,2013.0,Action,Tecmo Koei,0.05,0.05,0.25,0.03


In [3]:
# 수치 확인 및 변환

def fix(value):
  value = value.str.replace('M', '')
  value = value.str.replace('K', '0')
  value = pd.to_numeric(value)

  news = []
  for i in range(len(value)):

    if value[i] >= 100:
      x = value[i] / 10000
    else:
      x = value[i]

    news.append(x)
  return np.round(news, 3)

df1['NA_Sales'] = fix(df1['NA_Sales'])
df1['EU_Sales'] = fix(df1['EU_Sales'])
df1['JP_Sales'] = fix(df1['JP_Sales'])
df1['Other_Sales'] = fix(df1['Other_Sales']) 

df1['Total_Sales'] = ((df1['NA_Sales'] + df1['EU_Sales'] + df1['JP_Sales'] +df1['Other_Sales'])).round(3)# Total_Sales

In [9]:
# 종목이 없는 'etc'로 분류 한것을 클러스터링을 이용해 예측 

from sklearn.cluster import KMeans
import copy
pd.set_option('mode.chained_assignment',  None)


etc = df1[df1['Genre'] == 'etc']
etc.reset_index(inplace=True)
etc.drop(columns={'index', 'Name', 'Platform', 'Year', 'Genre', 'Publisher', 'Total_Sales'}, inplace=True)
etc_value = etc.values


model = KMeans(n_clusters=8, random_state=11).fit(etc_value)
x = model.labels_

df2 = df1[df1['Genre'] == 'etc']
df2['Genre'] = x

df3 = df2
df3['Genre'] = df3['Genre'].astype(str)

df3['Genre'] = df3['Genre'].replace('7', 'Action')
df3['Genre'] = df3['Genre'].replace('6', 'Sports')
df3['Genre'] = df3['Genre'].replace('5', 'Fighting')
df3['Genre'] = df3['Genre'].replace('4', 'Role-Playing')
df3['Genre'] = df3['Genre'].replace('3', 'Role-Playing')
df3['Genre'] = df3['Genre'].replace('2', 'Misc')
df3['Genre'] = df3['Genre'].replace('1', 'Role-Playing')
df3['Genre'] = df3['Genre'].replace('0', 'Adventure')

df2['Genre'] = df3['Genre']

df1[df1['Genre'] == 'etc'] = df2

df1

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Total_Sales
0,Candace Kane's Candy Factory,DS,2008.0,Action,Destineer,0.04,0.00,0.00,0.00,0.04
1,The Munchables,Wii,2009.0,Action,Namco Bandai Games,0.17,0.00,0.00,0.01,0.18
2,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010.0,Adventure,Alchemist,0.00,0.00,0.02,0.00,0.02
3,Deal or No Deal: Special Edition,DS,2010.0,Misc,Zoo Games,0.04,0.00,0.00,0.00,0.04
4,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010.0,Platform,D3Publisher,0.12,0.09,0.00,0.04,0.25
...,...,...,...,...,...,...,...,...,...,...
16593,Ice Age 2: The Meltdown,GC,2006.0,Platform,Vivendi Games,0.15,0.04,0.00,0.01,0.20
16594,Rainbow Islands: Revolution,PSP,2005.0,Action,Rising Star Games,0.01,0.00,0.00,0.00,0.01
16595,NBA 2K16,PS3,2015.0,Sports,Take-Two Interactive,0.44,0.19,0.03,0.13,0.79
16596,Toukiden: The Age of Demons,PSV,2013.0,Action,Tecmo Koei,0.05,0.05,0.25,0.03,0.38


In [11]:
 # 연도별 트렌드 분석 - 가장 많이 팔린 종류

df1['Year'] = pd.to_numeric(df1['Year'])
og_value = df1['Year'].values
og_values = []

# Year 이상치 변형

for i in range(len(og_value)):
  if og_value[i] <= 20:
    x = og_value[i] + 2000
  elif og_value[i] <= 100:
    x = og_value[i] + 1900
  else:
    x = og_value[i]
  og_values.append(x)

df1['Year'] = og_values 


og_tab = pd.crosstab(index=df1['Year'], columns=df1['Genre'], values=df1['Total_Sales'], aggfunc=sum)
og_tab.fillna(0, inplace=True)
og_ind = og_tab.index.values

def sales_year(x): # 년도 별 가장 높은 수 

  og_head = x.columns
  og_max_ind = []
  og_max_col = []
  og_max_value = []
  for i in range(len(x.index)):

    value = x.iloc[i].max()

    ind, col = np.where(x == x.iloc[i].max()) # 위치찾기 

    col, ind = col[0] ,ind[0]

    ogx, ogy, ogv = og_head[col], og_ind[ind] , value

    og_max_col.append(ogx)
    og_max_ind.append(ogy)
    og_max_value.append(ogv)
  return og_max_ind, og_max_col, og_max_value

max_year, max_genre, max_value = sales_year(og_tab)

import plotly.express as px

fig = px.bar(x= max_year, y=max_value, color=max_genre, text=np.round(max_value), height=400)
fig.update_layout(title="Sales rate by years")
fig.update_traces(textposition='outside')
fig.show()

In [12]:
no = pd.crosstab(index=df1['Platform'], columns=df1['Genre'], values=df1['EU_Sales'], aggfunc=sum)
head = no.columns # 헤드 만들기 

In [13]:
# 나라별 판매량

def preference(country):

  na = pd.crosstab(index=df1['Platform'], columns=df1['Genre'], values=country, aggfunc=sum)
  na.fillna(0, inplace=True)
  na1 = np.array(na.sum()).round()
  return na1# 평균

data = preference(df1['NA_Sales'])
country = pd.DataFrame(data=data, index=head, columns=['NA'])
country['EU'] = preference(df1['EU_Sales'])
country['JP'] = preference(df1['JP_Sales'])
country['os'] = preference(df1['Other_Sales'])
country = country.sort_values(ascending=False,axis=0, by='NA')

import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=country.index, y=country['NA'], text=country['NA'].values, textposition='top right',
                    mode='lines+markers+text',
                    name='NA'))
fig.add_trace(go.Scatter(x=country.index, y=country['EU'],text=country['EU'].values, textposition='top center',
                    mode='lines+markers+text',
                    name='EU'))
fig.add_trace(go.Scatter(x=country.index, y=country['JP'],text=country['JP'].values, textposition='bottom center',
                    mode='lines+markers+text',
                    name='JP'))
fig.add_trace(go.Scatter(x=country.index, y=country['os'],text=country['os'].values, textposition='bottom right',
                    mode='lines+markers+text',
                    name='OS'))
fig.update_layout(
    title="Sales rate by countries")

In [14]:
# 출고량 종류별 시각화 

def total(value):
  na = pd.crosstab(index=df1['Platform'], columns=df1['Genre'], values=value, aggfunc=sum)
  na.fillna(0, inplace=True)
  return na.round()

total = total(df1['Total_Sales']).sort_values(by='Action', ascending=False)

import plotly.graph_objects as go

labels = total.columns
values = total.sum().values

fig = go.Figure(data=[go.Pie(labels=labels, values=values ,title='Total Sales',pull=[0.2, 0, 0, 0])])
fig.update_traces(
    textposition = 'inside')
fig.update_layout(font=dict(size=18))

fig.show()

In [26]:
# 2010 년 이후에 출시된 게임중 매출액이 높은 게임들 - 19개

df1[(df1['Total_Sales'] >= 10) & (df1['Year'] >= 2010)].sort_values(ascending=False, by='Total_Sales').head(10)

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Total_Sales,new_year
4343,Kinect Adventures!,X360,2010.0,Misc,Microsoft Game Studios,14.97,4.94,0.24,1.67,21.82,2010
3483,Grand Theft Auto V,PS3,2013.0,Action,Take-Two Interactive,7.01,9.27,0.97,4.14,21.39,2010
10913,Grand Theft Auto V,X360,2013.0,Action,Take-Two Interactive,9.63,5.31,0.06,1.38,16.38,2010
10621,Pokemon Black/Pokemon White,DS,2010.0,Role-Playing,Nintendo,5.57,3.28,5.65,0.82,15.32,2010
2809,Call of Duty: Modern Warfare 3,X360,2011.0,Shooter,Activision,9.03,4.28,0.13,1.32,14.76,2010
6248,Call of Duty: Black Ops,X360,2010.0,Shooter,Activision,9.67,3.73,0.11,1.13,14.64,2010
1971,Pokemon X/Pokemon Y,3DS,2013.0,Role-Playing,Nintendo,5.17,4.05,4.34,0.79,14.35,2010
9070,Call of Duty: Black Ops 3,PS4,2015.0,Shooter,Activision,5.77,5.81,0.35,2.31,14.24,2010
4240,Call of Duty: Black Ops II,PS3,2012.0,Shooter,Activision,4.99,5.88,0.65,2.52,14.04,2010
6393,Call of Duty: Black Ops II,X360,2012.0,Shooter,Activision,8.25,4.30,0.07,1.12,13.74,2010


In [28]:
# 2000년 이후에 출시 된 게임중 매출액이 높은 게임들 
df1[(df1['Total_Sales'] >= 20) & (df1['Year'] >= 2000)].sort_values(ascending=False, by='Total_Sales').head(10)

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Total_Sales,new_year
2908,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74,2000
12579,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.83,2000
3352,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00,2000
6937,New Super Mario Bros.,DS,2006.0,Platform,Nintendo,11.38,9.23,6.50,2.90,30.01,2000
3014,Wii Play,Wii,2006.0,Misc,Nintendo,14.03,9.20,2.93,2.85,29.01,2000
16531,New Super Mario Bros. Wii,Wii,2009.0,Platform,Nintendo,14.59,7.06,4.70,2.26,28.61,2000
11137,Nintendogs,DS,2005.0,Simulation,Nintendo,9.07,11.00,1.93,2.75,24.75,2000
10650,Mario Kart DS,DS,2005.0,Racing,Nintendo,9.81,7.57,4.13,1.92,23.43,2000
3435,Wii Fit,Wii,2007.0,Sports,Nintendo,8.94,8.03,3.60,2.15,22.72,2000
10899,Wii Fit Plus,Wii,2009.0,Sports,Nintendo,9.09,8.59,2.53,1.79,22.00,2000


In [17]:
# 1995 , 2000(Nan 대체), 2010 으로 그룹화

df2 = df1

new = []
for i in range(len(df1['Year'])):

  if df2['Year'][i] >= 2010:
     x = 2010
  elif df2['Year'][i] >= 2000:
    x = 2000
  elif df2['Year'][i] <= 1950:
    x = 2000
  else: 
    x = 1990
  new.append(x)

df2['new_year'] = new 

In [18]:
# Rate by years , Trend 

two_five = df2[df2['new_year']== 2010]
two_zero = df2[df2['new_year']== 2000]
one_nine = df2[df2['new_year'] == 1990]

def years(year):
  na = pd.crosstab(index=df2['Platform'], columns=df2['Genre'], values=year, aggfunc=sum)
  na.fillna(0, inplace=True)
  na1 = np.array(na.sum()).round()
  return na1


df = years(one_nine['Total_Sales'])
year = pd.DataFrame(data=df, index=head, columns=['1990'])
# Data Frame 병합
year['2000'] = years(two_zero['Total_Sales'])
year['2010'] = years(two_five['Total_Sales'])
years = year.sort_values(ascending=False,axis=0, by='2010')

# 시각화 

from plotly.subplots import make_subplots


fig = make_subplots(rows=1, cols=1)

fig.add_scatter(y=years['2010'].values, x= years.index, mode="lines+markers+text", text=np.round(years['2010'].values, 2), textposition='top right',
                marker=dict(size=10, color="navy"),
                name='2010', row=1, col=1)

fig.add_scatter(y=years['2000'].values, x= years.index, mode="lines+text+markers", text=np.round(years['2000'].values, 2), textposition='middle right',
                marker=dict(size=10, color="LightSeaGreen"),
                name='2000', row=1, col=1)

fig.add_bar(y=years['1990'], x = years.index, text=np.round(years['1990'].values, 2), textposition='inside',
            marker=dict(color="MediumPurple"),
            name="1990", row=1, col=1)
fig.update_layout(
    title="Sales rate by years",
    xaxis_title="Genre",
    font=dict(size=16))

fig.show()

In [19]:
# 게임 생산 수와 판매량이 차이가 있는지 확인
# 1990 ~ 2000년, 2000 ~ 2009 년 , 2010 ~ 최근 비교 

x, y, z = np.sum(one_nine['Genre'].value_counts()), np.sum(two_zero['Genre'].value_counts()), np.sum(two_five['Genre'].value_counts())
count = [x, y, z] # 연도별 개수 파악

import plotly.graph_objects as graph_objects

fig = go.Figure(data=go.Bar(y =[x, y, z], x =[1990, 2000, 2010], text=count))
fig.update_traces(textposition='inside')
fig.update_layout(font=dict(size=18))

fig.show()

In [20]:
# ~2009년 , 2010년~  나라별 데이터 정리 

def sameyear(x, y):

  jp = pd.crosstab(index=two_five['Platform'], columns=two_five['Genre'], values=two_five[y], aggfunc=sum)
  jp.fillna(0, inplace=True)

  head = jp.columns
  two_five_sum = np.array(jp.sum())

  # 2009년까지 생산된 게임들 Data Frame

  df_sample = pd.DataFrame(data=two_five_sum, index=head, columns=['2010'])

  z = pd.crosstab(index = x['Platform'], columns=x['Genre'], values=x[y], aggfunc=sum)
  z.fillna(0, inplace=True)
  sum_ = np.array(z.sum()) 
                                        # 2010년 생산된 게임 수 / 2000 년도에 생산된 게임수 [ 발매한  게임 수를 같게 만듦 ]
  df_sample['2000'] = sum_
  return df_sample

# 나라별 데이터 정리

NA = sameyear(two_zero, 'NA_Sales')
JP = sameyear(two_zero, 'JP_Sales' )
EU = sameyear(two_zero, 'EU_Sales')
OS = sameyear(two_zero, 'Other_Sales')
name = NA.index

In [21]:
# (2000 ~ 2009년 과 2010 ~ 최근)  비교
# (2000~ 2009년 , 2010~) 최근  데이터를 시각화  
# 출시년도 대비 게임별 판매 정보 확보를 위해 

from plotly.subplots import make_subplots

fig = make_subplots(rows=2, cols=2)


fig.add_scatter(y=NA['2010'], x = name, mode="markers+text+lines", text=np.round(NA['2010'].values, 2), textposition="bottom center",
                marker=dict(size=10, color="green"),
                name="NA-2010", row=1, col=1)

fig.add_bar(y=NA['2000'], x = name,
            
            marker=dict(color="MediumPurple"),
            name="NA-2000", row=1, col=1)


fig.add_scatter(y=JP['2010'].values, x= name, mode="markers+text+lines", text=np.round(JP['2010'].values, 2), textposition="bottom center",
                marker=dict(size=10, color="red"),
                name='JP-2010', row=1, col=2)

fig.add_bar(y=JP['2000'], x=name,
            marker=dict(color="MediumPurple"),
            name="JP-2000", row=1, col=2)


fig.add_scatter(y=EU['2010'], x = name, mode="markers+text+lines", text=np.round(EU['2010'].values, 2), textposition="bottom center",
                marker=dict(size=10, color="navy"),
                name="EU-2010", row=2, col=1)

fig.add_bar(y=EU['2000'], x = name,
            marker=dict(color="MediumPurple"),
            name="EU-2000", row=2, col=1)


fig.add_scatter(y=OS['2010'], x = name, mode="markers+text+lines", text=np.round(OS['2010'].values, 2), textposition="bottom center",
                marker=dict(size=10, color="blue"),
                name="OS-2010", row=2, col=2)

fig.add_bar(y=OS['2000'], x = name,
            marker=dict(color="MediumPurple"),
            name="OS-2000", row=2, col=2)

fig.add_annotation(text="NA_Sales",
                  xref="paper", yref="paper",
                  x=0.15, y=0.95, showarrow=False)
fig.add_annotation(text="JP_Sales",
                  xref="paper", yref="paper",
                  x=0.75, y=0.95, showarrow=False)
fig.add_annotation(text="EU_Sales",
                  xref="paper", yref="paper",
                  x=0.15, y=0.35, showarrow=False)
fig.add_annotation(text="OS_Sales",
                  xref="paper", yref="paper",
                  x=0.75, y=0.3, showarrow=False)

fig.show()

In [ ]:
# 가설 
# 출시 된 연도 판매 수를 파악해 
# (2000~ 2009 : 2010 ~ 최근) 판매 수를 맞춰 
# 그래프를 통해 나라별 최근 게임 선호도 및 구매 추세를 확인 할 수 있다. 
# 예측 - 게임 발매량이 많아지면 매출액이 상승한다?  =  연관성이 없진 않다.
count

[1960, 9526, 5112]

In [22]:
# 2000 ~ 2009년 , 2010 ~ 현재 게임 발매 수 비교 

from collections import Counter
counter = Counter(two_five['Genre'])
counter0 = Counter(two_zero['Genre'])

counter_head, counter_value = list(counter.keys()), list(counter.values())
counter0_head, counter0_value = list(counter0.keys()), list(counter0.values())
counter0_value

import plotly.graph_objects as go

genre0 = counter0_head
genre = counter_head

fig = go.Figure()

fig.add_trace(go.Bar(x=genre,
                y=counter_value,
                name='2010 ~',
                marker_color='rgb(26, 118, 255)',
                text=counter_value
                ))
fig.add_trace(go.Bar(x=genre0,
                y=counter0_value,
                name='2000~2009',
                marker_color='rgb(55, 83, 109)',
                text=counter0_value
                ))

fig.update_layout( 
    title='Number of releases',
    xaxis_tickfont_size=14,

    legend=dict(
        x=0.9,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.15, 
    bargroupgap=0.1)
fig.update_traces(textposition='inside')
fig.update_layout(font=dict(size=10))

fig.show()

In [23]:
# 2010 ~ Now , 2000 ~ 2009 년 트랜드 방향 

two_10 = pd.crosstab(index=two_five['Platform'], columns=two_five['Genre'], values=two_five['Total_Sales'], aggfunc=sum)
two_10.fillna(0, inplace=True)
two_00 = pd.crosstab(index=two_zero['Platform'], columns=two_zero['Genre'], values=two_zero['Total_Sales'], aggfunc=sum)
two_00.fillna(0, inplace=True)


from plotly.subplots import make_subplots


fig = make_subplots(3, 2)


fig.add_bar(x=two_10.index,
            y=two_10['Action'],
            text= 'Action',
            name='2010 ~ Now Platform ',
            marker_color='rgb(26, 118, 255)',
            row=1, col=1
            )

fig.add_bar(x=two_00.index,
            y=two_00['Action'],
            text= 'Action',
            name='2000 ~ 2009 ',
            
            row=1, col=2
            )

fig.add_bar(x=two_10.index,
            y=two_10['Shooter'],
            text= 'Shooter',
            name='2010 ~ Now Platform ',
            marker_color='rgb(26, 118, 255)',
            row=2, col=1
            )
fig.add_bar(x=two_00.index,
            y=two_00['Shooter'],
            text= 'Shooter',
            name='2000 ~ 2009 ',
            
            row=2, col=2
            )
fig.add_bar(x=two_10.index,
            y=two_10['Role-Playing'],
            text= 'Role-Playing',
            name='2010 ~ Now Platform ',
            marker_color='rgb(26, 118, 255)',
            row=3, col=1
            )
fig.add_bar(x=two_00.index,
            y=two_00['Role-Playing'],
            text= 'Role-Playing',
            name='2000 ~ 2009 ',
            
            row=3, col=2
            )
fig.add_annotation(text='2010 ~ Now Action ',
                  xref="paper", yref="paper",
                  x=0.2, y=1.0, showarrow=False)

fig.add_annotation(text='2000 ~ 2009  Action',
                  xref="paper", yref="paper",
                  x=0.7, y=1.0, showarrow=False)

fig.add_annotation(text='2010 ~ Now Shooter ',
                  xref="paper", yref="paper",
                  x=0.2, y=0.6, showarrow=False)

fig.add_annotation(text='2000 ~ 2009 Shooter',
                  xref="paper", yref="paper",
                  x=0.7, y=0.60, showarrow=False)

fig.add_annotation(text='2010 ~ Now Role-playing ',
                  xref="paper", yref="paper",
                  x=0.2, y=0.2, showarrow=False)

fig.add_annotation(text='2000 ~ 2009  Role-playing',
                  xref="paper", yref="paper",
                  x=0.74, y=0.2, showarrow=False)



fig.show()

In [144]:
# 나라별 2010년 이후 플랫폼 판매량

def platform(z):
  x = pd.crosstab(index=two_five['Genre'], columns=two_five['Platform'], values=two_five[z], aggfunc=sum)
  x.fillna(0, inplace=True)
  
  y = pd.crosstab(index=two_zero['Genre'], columns=two_zero['Platform'], values=two_zero[z], aggfunc=sum)
  y.fillna(0, inplace=True)
 

  return x, y
na_sales, na_sales_0 = platform('NA_Sales')
eu_sales, eu_sales_0 = platform('EU_Sales')
jp_sales, jp_sales_0 = platform('JP_Sales')
os_sales, os_sales_0 = platform('Other_Sales')

title = ['3DS','DS','PC',	'PS2',	'PS3',	'PS4',	'PSP',	'PSV',	'Wii',	'WiiU','X360','XOne']
title_0 = ['2600', '3DS', 'DC', 'DS', 'GB', 'GBA', 'GC', 'N64', 'PC', 'PS', 'PS2', 'PS3', 'PSP', 'PSV', 'WS', 'Wii', 'X360', 'XB']


fig = go.Figure()
fig.add_trace(go.Bar(x=title,
                y=na_sales.sum().values,
                name='NA_2010~',
                marker_color='navy'
                ))
fig.add_trace(go.Bar(x=title,
                y=eu_sales.sum().values,
                name='EU_2010~',
                marker_color='purple'
                ))
fig.add_trace(go.Bar(x=title,
                y=jp_sales.sum().values,
                name='JP_2010~',
                marker_color='green'
                ))
fig.add_trace(go.Bar(x=title,
                y=os_sales.sum().values,
                name='Other_2010~',
                marker_color='black'
                ))
fig.update_traces(textposition='inside')
fig.update_layout(font=dict(size=20))
fig.update_layout(barmode='group')
fig.show()

In [143]:
# 나라별 2000 ~ 2009년 플랫폼 판매량
title_0 = ['2600', '3DS', 'DC', 'DS', 'GB', 'GBA', 'GC', 'N64', 'PC', 'PS', 'PS2', 'PS3', 'PSP', 'PSV', 'WS', 'Wii', 'X360', 'XB']


fig = go.Figure()
fig.add_trace(go.Bar(x=title_0,
                y=na_sales_0.sum().values,
                name='NA_2000~2009',
                marker_color='navy'
                ))
fig.add_trace(go.Bar(x=title_0,
                y=eu_sales_0.sum().values,
                name='EU_2000~2009',
                marker_color='purple'
                ))
fig.add_trace(go.Bar(x=title_0,
                y=jp_sales_0.sum().values,
                name='JP_2000~ 2009',
                marker_color='green'
                ))
fig.add_trace(go.Bar(x=title_0,
                y=os_sales_0.sum().values,
                name='Other_2000~ 2009',
                marker_color='black'
                ))
fig.update_traces(textposition='inside')
fig.update_layout(font=dict(size=20))
fig.update_layout(barmode='group')
fig.show()

In [138]:
na_sales_0.sum()

Platform
2600      9.82
3DS       0.84
DC        2.53
DS      327.84
GB        9.80
GBA     187.54
GC      133.46
N64      21.99
PC       25.88
PS       67.50
PS2     581.52
PS3     164.15
PSP      99.41
PSV       0.13
WS        0.00
Wii     381.20
X360    268.13
XB      186.69
dtype: float64

In [24]:
# 2010년 이후 게임 발매 판매량 

import plotly.graph_objects as go

def total(x, value):

  na = pd.crosstab(index=x['Platform'], columns=x['Genre'], values=x[value], aggfunc=sum)
  na.fillna(0, inplace=True)
  return na.round()

total1 = total(two_five, 'Total_Sales')
labels = total1.columns
values = total1.sum().values

fig = go.Figure(data=[go.Pie(labels=labels, values=values ,title='2010 ~ Now Total Sales',pull=[0.2, 0, 0, 0])])
fig.update_traces(
    textposition = 'inside')
fig.update_layout(font=dict(size=18))
fig.update_traces(textposition='inside', textinfo='percent+label')

fig.show()


In [25]:
# 2000~ 2009 년 게임 발매 판매량

total0 = total(two_zero, 'Total_Sales')
labels0 = total0.columns
values0 = total0.sum().values

fig = go.Figure(data=[go.Pie(labels=labels0, values=values0 ,title='2000 ~ 2009 Total Sales',pull=[0.2, 0, 0, 0])])
fig.update_traces(
    textposition = 'inside')
fig.update_layout(font=dict(size=18))
fig.update_traces(textposition='inside', textinfo='percent+label')

fig.show()
